In [ ]:
pub_1050554453b1c0327cbb09763d1c227223e8a

In [6]:
from newsdataapi import NewsDataApiClient

api = NewsDataApiClient(apikey="")

response = api.news_api(country = "gb", language="en")
articles = response.get("results")  # list of dicts
id = res.get("link")
text = res.get("content")

for article in articles:
    yield article.get("link"), res.get("content")

[{'title': 'Games with Gold September 2022 Reveal Schedule & Xbox Release Date',
  'link': 'https://www.hitc.com/en-gb/2022/08/22/games-with-gold-september-2022-xbox-release-date/?utm_source=rss&utm_medium=rss&utm_campaign=games-with-gold-september-2022-xbox-release-date',
  'keywords': ['Gaming', 'Gaming News', 'Games With Gold'],
  'creator': ['Gary Jones'],
  'video_url': None,
  'description': 'Microsoft will reveal its plans for Games with Gold September 2022 before the end of the current month, including the final Xbox 360 titles that... The post Games with Gold September 2022 Reveal Schedule & Xbox Release Date appeared first on HITC.',
  'content': 'Microsoft will reveal its plans for Games with Gold September 2022 before the end of the current month, including the final Xbox 360 titles that will be given away for free. September looks set to be the final month that fans receive four free games instead of two, and it’s unclear if the tech giant plans on changing anything else w

In [70]:
# !pip install transformers
# !pip install pandas
# !pip install sklearn
!pip install Scikit-multilearn

     |████████████████████████████████| 92kB 5.9MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import os
import pandas as pd
  
### Get raw_data_labeled DF

data_dir = "./data"
   
def get_label_and_text(path):
    '''
    Path should have folders indicating labels 
    with sorted .txt files containing text data.
    
    '''
    for subdir, dirs, files in os.walk(path):
        for file in files:
            parent_dir = os.path.basename(subdir)
            file_path = os.path.join(subdir, file)
            if not file_path.endswith('txt'):
                continue
            with open(file_path) as f:
                text = f.read()
                yield (parent_dir, text)
    

labeled_data_generator = get_label_and_text(data_dir)
raw_data_labeled = pd.DataFrame(labeled_data_generator, columns=["label", "text_raw"])

print(len(raw_data_labeled))  # 2225

raw_data_labeled.head()

2225


,label,text_raw
0,entertainment,Musicians to tackle US red tape\n\nMusicians' ...
1,entertainment,"U2's desire to be number one\n\nU2, who have w..."
2,entertainment,Rocker Doherty in on-stage fight\n\nRock singe...
3,entertainment,Snicket tops US box office chart\n\nThe film a...
4,entertainment,Ocean's Twelve raids box office\n\nOcean's Twe...


In [2]:
df = raw_data_labeled
df['text_raw']

0       Musicians to tackle US red tape\n\nMusicians' ...
1       U2's desire to be number one\n\nU2, who have w...
2       Rocker Doherty in on-stage fight\n\nRock singe...
3       Snicket tops US box office chart\n\nThe film a...
4       Ocean's Twelve raids box office\n\nOcean's Twe...
                              ...                        
2220    Warning over Windows Word files\n\nWriting a M...
2221    Fast lifts rise into record books\n\nTwo high-...
2222    Nintendo adds media playing to DS\n\nNintendo ...
2223    Fast moving phone viruses appear\n\nSecurity f...
2224    Hacker threat to Apple's iTunes\n\nUsers of Ap...
Name: text_raw, Length: 2225, dtype: object

In [52]:
# !pip install nltk
nltk.download('stopwords')

NameError: name 'nltk' is not defined

In [53]:
import nltk
# nltk.download()
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/Maria/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [90]:
### CLEAN AND TOKENIZE
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string

def clean_text(column):
    column = column.str.lower()
    column = column.str.replace(f'[{string.punctuation}]', '', regex=True)
    column = column.str.replace('\n',' ', regex=True)
    column = column.apply(word_tokenize)
    stop_words = set(stopwords.words('english'))
    column = column.apply(lambda words: [w for w in words if not w in stop_words])
    column = column.apply(lambda words: [PorterStemmer().stem(word) for word in words]) 
    return column
                          
df['text_clean'] = clean_text(df['text_raw'])
df.head()

,label,text_raw,text_clean,label_nums
0,entertainment,Musicians to tackle US red tape\n\nMusicians' ...,"['musician', 'tackl', 'us', 'red', 'tape', 'mu...",1
1,entertainment,"U2's desire to be number one\n\nU2, who have w...","['u2', 'desir', 'number', 'one', 'u2', 'three'...",1
2,entertainment,Rocker Doherty in on-stage fight\n\nRock singe...,"['rocker', 'doherti', 'onstag', 'fight', 'rock...",1
3,entertainment,Snicket tops US box office chart\n\nThe film a...,"['snicket', 'top', 'us', 'box', 'offic', 'char...",1
4,entertainment,Ocean's Twelve raids box office\n\nOcean's Twe...,"['ocean', 'twelv', 'raid', 'box', 'offic', 'oc...",1


In [91]:
# Encode labels
from sklearn.preprocessing import LabelEncoder
df["label_nums"] = LabelEncoder().fit_transform(df["label"])
df.tail()


df["label_nums"].nunique()

X = df["text_clean"]
y = df["label_nums"]
print(X)
print(y)

0       ['musician', 'tackl', 'us', 'red', 'tape', 'mu...
1       ['u2', 'desir', 'number', 'one', 'u2', 'three'...
2       ['rocker', 'doherti', 'onstag', 'fight', 'rock...
3       ['snicket', 'top', 'us', 'box', 'offic', 'char...
4       ['ocean', 'twelv', 'raid', 'box', 'offic', 'oc...
                              ...                        
2220    ['warn', 'window', 'word', 'file', 'write', 'm...
2221    ['fast', 'lift', 'rise', 'record', 'book', 'tw...
2222    ['nintendo', 'add', 'media', 'play', 'ds', 'ni...
2223    ['fast', 'move', 'phone', 'virus', 'appear', '...
2224    ['hacker', 'threat', 'appl', 'itun', 'user', '...
Name: text_clean, Length: 2225, dtype: object
0       1
1       1
2       1
3       1
4       1
       ..
2220    4
2221    4
2222    4
2223    4
2224    4
Name: label_nums, Length: 2225, dtype: int64


pandas.core.series.Series

In [92]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, stratify=y)

X_test

hi


1192    ['wolv', 'appoint', 'hoddl', 'manag', 'glenn',...
22      ['produc', 'scoop', 'stage', 'award', 'produc'...
1539    ['chancellor', 'ralli', 'labour', 'voter', 'go...
1965    ['long', 'life', 'promis', 'laptop', 'pc', 'sc...
254     ['veteran', 'comic', 'cyril', 'fletcher', 'die...
                              ...                        
806     ['diageo', 'buy', 'us', 'wine', 'firm', 'diage...
750     ['takeov', 'rumour', 'lift', 'exel', 'share', ...
209     ['aviat', 'vera', 'take', 'bafta', 'glori', 'h...
1320    ['highburi', 'tunnel', 'player', 'clear', 'foo...
1568    ['crucial', 'decis', 'supercasino', 'decis', '...
Name: text_clean, Length: 668, dtype: object

In [104]:
## Feature extraction = Vectorization and Train

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

nb = Pipeline([('tfidf', TfidfVectorizer()),
               ('clf', MultinomialNB()),
              ])
              
nb.fit(X_train,y_train)

test_predict = nb.predict(X_test)

train_accuracy = round(nb.score(X_train,y_train)*100)
test_accuracy =round(accuracy_score(test_predict, y_test)*100)



In [105]:
print(train_accuracy)
print(test_accuracy)

99
97


In [9]:
import sys
print(sys.executable)

/usr/local/opt/python@3.8/bin/python3.8


In [11]:
import sys

sys.version

'3.8.3 (default, Jul  8 2020, 14:27:55) \n[Clang 11.0.3 (clang-1103.0.32.62)]'

In [ ]:
# pip install -q transformers
print("hi")
from transformers import pipeline
sentiment_pipeline = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")
data = ["I love you", "I hate you"]
sentiment_pipeline(data)

In [ ]:
!pip install torch

     |████████████████████████████████| 137.6MB 58kB/s  eta 0:00:01


In [ ]:
!pip install transformers